In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import zipfile

from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST


import flwr as fl
from flwr.common import Metrics

import numpy as np
import pandas as pd
import random
import cv2
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score



In [ ]:
CLASSES = (
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10

# IID


In [ ]:
import numpy as np
from sklearn.decomposition import PCA

def determine_pca_components(public_dataset, n_components=64):
    # Reshape the public dataset to 2D (num_samples, num_features)
    public_data = public_dataset.reshape(public_dataset.shape[0], -1)

    # Fit PCA on the public dataset
    pca = PCA(n_components=n_components)
    pca.fit(public_data)

    return pca

In [ ]:
def apply_pca_locally(data, pca, n_components):
    # Reshape the data to 2D (num_samples, num_features)
    data_2d = data.reshape(data.shape[0], -1)

    # Apply PCA transformation
    data_transformed = pca.transform(data_2d)[:, :n_components]

    # Calculate the percentage of data reduced or removed
    original_size = data_2d.size
    transformed_size = data_transformed.size
    data_removed_percentage = (1 - transformed_size / original_size) * 100

    # Reshape the transformed data to (num_samples, 1, n_components)
    data_transformed = data_transformed.reshape(-1, 1, n_components)

    return data_transformed, data_removed_percentage

In [ ]:
from torchvision.datasets import MNIST

# Define the transform to preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_dataset = MNIST("./dataset", train=True, download=True, transform=transform)

# Load the MNIST dataset
trainset = MNIST(root='./data', train=True, download=True, transform=transform)
testset = MNIST(root='./data', train=False, download=True, transform=transform)


100%|██████████| 9912422/9912422 [00:00<00:00, 90418541.51it/s]


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 20331603.53it/s]

Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 33266110.92it/s]


...

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33001805.66it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12871978.90it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
pca_components_range = [8, 16, 32, 64, 128]
#pca_components_range = [16, 32]

In [ ]:
BATCH_SIZE = 32

import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, random_split, Subset

def load_datasets(pca, n_components):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    trainset = MNIST("./dataset", train=True, download=True, transform=transform)
    testset = MNIST("./dataset", train=False, download=True, transform=transform)

    trainset_transformed, train_removed_percentage = apply_pca_locally(trainset.data.numpy(), pca, n_components)
    testset_transformed, test_removed_percentage = apply_pca_locally(testset.data.numpy(), pca, n_components)

    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10
        len_train = len(ds) - len_val
        indices = list(range(len(ds)))
        train_indices = indices[:len_train]
        val_indices = indices[len_train:]
        ds_train = Subset(ds, train_indices)
        ds_val = Subset(ds, val_indices)
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader, datasets, testset, train_removed_percentage, test_removed_percentage


we can can two major insights from this graph:

a. Based on the bar graph image you've provided, the distribution appears to be homogeneous. Each client has data across all the different classes (digits 0-9), as indicated by the presence of different colors (representing different clients) within each digit's bar.

b. The graph and the code imply an IID distribution. In the context of the MNIST dataset, each client's data is a random subset of the entire dataset, maintaining the overall distribution of labels. The IID distribution is supported by the fact that each digit is represented fairly evenly across all clients, as shown by the similar heights of different colors within each bar on the graph. Non-IID would show a significant imbalance, with some clients potentially having more of certain digits and less of others.




## Centralized Training with pytorch

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
DEVICE = torch.device("cpu")

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.optim import Adam
from torch.nn.functional import relu, softmax
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from sklearn.decomposition import PCA

def train(net, trainloader, validloader, testloader, epochs, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    best_valid_loss = float('inf')
    best_model = None
    best_test_loss = float('inf')
    best_test_accuracy = 0.0
    patience_counter = 0
    early_stopping_epoch = 0

    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_acc = correct / total

        net.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in validloader:
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        valid_loss = running_loss / len(validloader)
        valid_acc = correct / total

        # Calculate test loss and accuracy
        with torch.no_grad():
            test_loss, test_accuracy = test(net, testloader)

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")

        scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_model = net.state_dict()
            best_test_loss = test_loss
            best_test_accuracy = test_accuracy
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= 10:
            print(f"Early stopping at epoch {epoch+1}")
            early_stopping_epoch = epoch + 1
            break

    net.load_state_dict(best_model)
    return net, early_stopping_epoch, best_test_loss, best_test_accuracy

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy


In [ ]:
#pca_components_range = [16, 32]  # Define the range of PCA components to try
pca_components_range = [8, 16, 32, 64, 128]

best_accuracy = 0
best_pca_components = None

for n_components in pca_components_range:
    print(f"Training with {n_components} PCA components...")

    pca = determine_pca_components(mnist_dataset.data.numpy(), n_components=n_components)
    trainloaders, valloaders, testloader, datasets, testset, _, _ = load_datasets(pca, n_components)

    train_removed_percentages = []
    test_removed_percentages = []
    for i in range(NUM_CLIENTS):
        _, train_removed_percentage = apply_pca_locally(datasets[i].dataset.data.numpy(), pca, n_components)
        _, test_removed_percentage = apply_pca_locally(testset.data.numpy(), pca, n_components)
        train_removed_percentages.append(train_removed_percentage)
        test_removed_percentages.append(test_removed_percentage)

    trainloader = trainloaders[0]
    valloader = valloaders[0]
    net = Net().to(DEVICE)

    # Training and evaluation loop
    net, early_stopping_epoch, test_loss, test_accuracy = train(net, trainloader, valloader, testloader, epochs=100, device=DEVICE)
    print(f"Final test set performance for {n_components} PCA components: Loss {test_loss:.4f}, Accuracy {test_accuracy:.4f}")
    print(f"Early stopping at epoch: {early_stopping_epoch}")
    print()

Training with 8 PCA components...
Epoch 1/100 - Train Loss: 0.8130, Train Acc: 0.7415, Valid Loss: 0.2556, Valid Acc: 0.9133, Test Loss: 0.0080, Test Acc: 0.9272
Epoch 2/100 - Train Loss: 0.2272, Train Acc: 0.9291, Valid Loss: 0.1621, Valid Acc: 0.9517, Test Loss: 0.0043, Test Acc: 0.9613
Epoch 3/100 - Train Loss: 0.1529, Train Acc: 0.9531, Valid Loss: 0.1085, Valid Acc: 0.9733, Test Loss: 0.0033, Test Acc: 0.9647
Epoch 4/100 - Train Loss: 0.1033, Train Acc: 0.9654, Valid Loss: 0.1089, Valid Acc: 0.9733, Test Loss: 0.0030, Test Acc: 0.9694
Epoch 5/100 - Train Loss: 0.0770, Train Acc: 0.9750, Valid Loss: 0.0960, Valid Acc: 0.9733, Test Loss: 0.0024, Test Acc: 0.9759
Epoch 6/100 - Train Loss: 0.0547, Train Acc: 0.9830, Valid Loss: 0.1007, Valid Acc: 0.9783, Test Loss: 0.0023, Test Acc: 0.9772
Epoch 7/100 - Train Loss: 0.0439, Train Acc: 0.9856, Valid Loss: 0.1127, Valid Acc: 0.9683, Test Loss: 0.0028, Test Acc: 0.9714
Epoch 8/100 - Train Loss: 0.0362, Train Acc: 0.9881, Valid Loss: 0.109

## Federated Learning with Flower

This means that multiple clients (simulating a distributed data scenario) will train a neural network (Net) on their local datasets and send the updated parameters to a central server. The server then averages these parameters and distributes the result back to the clients for further training.


In [ ]:
import io

# Model size calculation function
def get_model_size(model):
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size = buffer.tell()  # Size in bytes
    return size

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import time
import numpy as np
from collections import OrderedDict
import torch

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})

    # Print the expected parameter shapes and the received parameter shapes
    print("Expected parameter shapes:")
    for name, param in net.named_parameters():
        print(f"{name}: {param.shape}")

    print("Received parameter shapes:")
    for name, param in state_dict.items():
        print(f"{name}: {param.shape}")

    # Try to load the state dictionary and handle shape mismatch
    try:
        net.load_state_dict(state_dict, strict=True)
    except RuntimeError as e:
        print("Shape mismatch occurred. Attempting to handle gracefully...")

        # Create a new state dictionary with matching shapes
        new_state_dict = OrderedDict()
        for (name, param), (state_name, state_param) in zip(net.named_parameters(), state_dict.items()):
            if param.shape != state_param.shape:
                print(f"Shape mismatch for parameter '{name}'. Expected {param.shape}, got {state_param.shape}.")
                if param.numel() == state_param.numel():
                    # Reshape the parameter if the total number of elements matches
                    state_param = state_param.view(param.shape)
                else:
                    print(f"Cannot reshape parameter '{name}' due to incompatible number of elements.")
                    continue
            new_state_dict[name] = state_param

        # Load the new state dictionary
        net.load_state_dict(new_state_dict, strict=False)
        print("Shape mismatch handled.")




In [ ]:
import time
import numpy as np
from collections import OrderedDict
import torch
import flwr as fl

DEVICE = torch.device("cpu")

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, testloader, pca, train_removed_percentage, test_removed_percentage, early_stopping_epoch, n_components):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.testloader = testloader  # Add testloader attribute
        self.pca = pca
        self.train_removed_percentage = train_removed_percentage
        self.test_removed_percentage = test_removed_percentage
        self.early_stopping_epoch = early_stopping_epoch
        self.n_components = n_components
        self.original_model_size = get_model_size(net)
        self.communication_rounds = 0
        self.communication_cost = 0
        self.latency = 0
        self.test_accuracy = 0.0
        self.test_loss = 0.0

    def get_parameters(self, **kwargs):
        return get_parameters(self.net)

    def set_parameters(self, parameters):
        set_parameters(self.net, parameters)

    def fit(self, parameters, config):
        start_time = time.time()
        self.set_parameters(parameters)
        self.test_loss, self.test_accuracy = test(self.net, self.testloader)
        epochs = config.get("epochs", self.early_stopping_epoch)  # Use early_stopping_epoch as default if 'epochs' key is not present
        self.net, _, _, _ = train(self.net, self.trainloader, self.valloader, self.testloader, epochs, DEVICE)

        self.communication_rounds += 1
        self.communication_cost += get_model_size(self.net) * (1 - self.train_removed_percentage / 100)
        self.latency += (time.time() - start_time) * (1 - self.train_removed_percentage / 100)

        return self.get_parameters(), len(self.trainloader.dataset), {
            "pca_reduction_percentage": self.train_removed_percentage,
            "n_components": self.n_components,
            "communication_rounds": self.communication_rounds,
            "communication_cost": self.communication_cost,
            "test_accuracy": self.test_accuracy,
            "test_loss": self.test_loss,
            "latency": self.latency,
        }

    def evaluate(self, parameters, config):
        start_time = time.time()
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.valloader)

        self.communication_rounds += 1
        self.communication_cost += get_model_size(self.net) * (1 - self.test_removed_percentage / 100)
        self.latency += (time.time() - start_time) * (1 - self.test_removed_percentage / 100)

        return float(loss), len(self.valloader.dataset), {
            "accuracy": float(accuracy),
            "pca_reduction_percentage": float(self.test_removed_percentage),
            "n_components": float(self.n_components),
            "communication_rounds": float(self.communication_rounds),
            "communication_cost": float(self.communication_cost),
            "latency": float(self.latency),
        }



In [ ]:
def client_fn(cid, pca, train_removed_percentage, test_removed_percentage, testloader, early_stopping_epoch, n_components):
    model = Net().to(DEVICE)
    trainloaders, valloaders, _, client_datasets, _, _, _ = load_datasets(pca, n_components)
    client = FlowerClient(model, trainloaders[int(cid)], valloaders[int(cid)], testloader, pca, train_removed_percentage, test_removed_percentage, early_stopping_epoch, n_components)
    return client.to_client()


In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m.get("accuracy", 0.0) for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    pca_reduction_percentages = [m.get("pca_reduction_percentage", 0.0) for _, m in metrics]
    communication_rounds = [m.get("communication_rounds", 0.0) for _, m in metrics]
    communication_cost = [m.get("communication_cost", 0.0) for _, m in metrics]
    test_accuracies = [m.get("test_accuracy", 0.0) for _, m in metrics]
    test_losses = [m.get("test_loss", 0.0) for _, m in metrics]
    latency = [m.get("latency", 0.0) for _, m in metrics]

    return {
        "accuracy": sum(accuracies) / sum(examples) if examples else 0.0,
        "pca_reduction_percentage": sum(pca_reduction_percentages) / len(pca_reduction_percentages) if pca_reduction_percentages else 0.0,
        "communication_rounds": sum(communication_rounds) / len(communication_rounds) if communication_rounds else 0.0,
        "communication_cost": sum(communication_cost),
        "test_accuracy": sum(test_accuracies) / len(test_accuracies) if test_accuracies else 0.0,
        "test_loss": sum(test_losses) / len(test_losses) if test_losses else 0.0,
        "latency": sum(latency) / len(latency) if latency else 0.0,
    }

In [ ]:

for n_components in pca_components_range:
    print(f"Training with {n_components} PCA components...")

    pca = determine_pca_components(mnist_dataset.data.numpy(), n_components=n_components)
    trainloaders, valloaders, testloader, datasets, testset, _, _ = load_datasets(pca, n_components)

    train_removed_percentages = []
    test_removed_percentages = []
    for i in range(NUM_CLIENTS):
        _, train_removed_percentage = apply_pca_locally(datasets[i].dataset.data.numpy(), pca, n_components)
        _, test_removed_percentage = apply_pca_locally(testset.data.numpy(), pca, n_components)
        train_removed_percentages.append(train_removed_percentage)
        test_removed_percentages.append(test_removed_percentage)

    model = Net().to(DEVICE)
    trainloader = trainloaders[0]
    valloader = valloaders[0]
    model, early_stopping_epoch, test_loss, test_accuracy = train(model, trainloader, valloader, testloader, epochs=100, device=DEVICE)

    print(f"Early stopping epoch for {n_components} PCA components: {early_stopping_epoch}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")


    def client_fn_with_pca(cid):
        return client_fn(cid, pca, train_removed_percentages[int(cid)], test_removed_percentages[int(cid)], testloader, early_stopping_epoch, n_components)

    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=10,
        min_evaluate_clients=5,
        min_available_clients=10,
        evaluate_metrics_aggregation_fn=weighted_average,
    )

    try:
        history = fl.simulation.start_simulation(
            client_fn=client_fn_with_pca,
            num_clients=NUM_CLIENTS,
            config=fl.server.ServerConfig(num_rounds=early_stopping_epoch),
            strategy=strategy,
        )
    except Exception as e:
        print(f"Simulation failed with error: {str(e)}")
        continue  # Continue with the next iteration if simulation fails

    try:
        if history.metrics_distributed and history.metrics_distributed[-1]:
            final_metrics = history.metrics_distributed[-1][-1]
            print(f"Metrics at early stopping epoch {early_stopping_epoch} for {n_components} PCA components:")
            print(f"  Accuracy: {final_metrics['accuracy']:.4f}")
            print(f"  PCA Reduction Percentage: {final_metrics['pca_reduction_percentage']:.2f}%")
            print(f"  Communication Rounds: {final_metrics['communication_rounds']:.2f}")
            print(f"  Communication Cost: {final_metrics['communication_cost']:.2f} bytes")
            print(f"  Test Loss: {final_metrics['test_loss']:.4f}")
            print(f"  Test Accuracy: {final_metrics['test_accuracy']:.4f}")
            print(f"  Latency: {final_metrics['latency']:.2f} seconds")
        else:
            print("No metrics available.")
    except (KeyError, IndexError) as e:
        print(f"Error accessing metrics_distributed: {str(e)}")

    # Print the contents of history.metrics_distributed for debugging
    print("Contents of history.metrics_distributed:")
    print(history.metrics_distributed)

# Calculate average metrics for each PCA component

# Calculate average metrics for each PCA component
for n_components in pca_components_range:
    print(f"Metrics for {n_components} PCA components:")
    accuracies = []
    test_losses = []
    test_accuracies = []
    communication_costs = []
    latencies = []

    for epoch in range(1, early_stopping_epoch + 1):
        if epoch <= len(history.metrics_distributed):
            epoch_metrics = history.metrics_distributed[epoch - 1]
            if epoch_metrics:
                epoch_accuracies = [client[1].get("accuracy", 0.0) for client in epoch_metrics]
                accuracies.extend(epoch_accuracies)

                epoch_test_losses = [client[1].get("test_loss", 0.0) for client in epoch_metrics]
                test_losses.extend(epoch_test_losses)

                epoch_test_accuracies = [client[1].get("test_accuracy", 0.0) for client in epoch_metrics]
                test_accuracies.extend(epoch_test_accuracies)

                epoch_communication_costs = [client[1].get("communication_cost", 0.0) for client in epoch_metrics]
                communication_costs.extend(epoch_communication_costs)

                epoch_latencies = [client[1].get("latency", 0.0) for client in epoch_metrics]
                latencies.extend(epoch_latencies)

    if accuracies:
        avg_accuracy = sum(accuracies) / len(accuracies)
        avg_test_loss = sum(test_losses) / len(test_losses)
        avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)
        avg_communication_cost = sum(communication_costs) / len(communication_costs)
        avg_latency = sum(latencies) / len(latencies)

        print(f"Average metrics for {n_components} PCA components (up to early stopping epoch {early_stopping_epoch}):")
        print(f"  Average Accuracy: {avg_accuracy:.4f}")
        print(f"  Average Test Loss: {avg_test_loss:.4f}")
        print(f"  Average Test Accuracy: {avg_test_accuracy:.4f}")
        print(f"  Average Communication Cost: {avg_communication_cost:.2f} bytes")
        print(f"  Average Latency: {avg_latency:.2f} seconds")
        print()
    else:
        print(f"No metrics available for {n_components} PCA components.")
        print()

Training with 8 PCA components...
Epoch 1/100 - Train Loss: 0.8781, Train Acc: 0.7159, Valid Loss: 0.2619, Valid Acc: 0.9300, Test Loss: 0.0086, Test Acc: 0.9281
Epoch 2/100 - Train Loss: 0.2685, Train Acc: 0.9170, Valid Loss: 0.1649, Valid Acc: 0.9550, Test Loss: 0.0045, Test Acc: 0.9592
Epoch 3/100 - Train Loss: 0.1616, Train Acc: 0.9506, Valid Loss: 0.1327, Valid Acc: 0.9683, Test Loss: 0.0033, Test Acc: 0.9663
Epoch 4/100 - Train Loss: 0.1126, Train Acc: 0.9619, Valid Loss: 0.1078, Valid Acc: 0.9717, Test Loss: 0.0031, Test Acc: 0.9690
Epoch 5/100 - Train Loss: 0.0966, Train Acc: 0.9691, Valid Loss: 0.1063, Valid Acc: 0.9717, Test Loss: 0.0026, Test Acc: 0.9733
Epoch 6/100 - Train Loss: 0.0737, Train Acc: 0.9776, Valid Loss: 0.0933, Valid Acc: 0.9833, Test Loss: 0.0024, Test Acc: 0.9772
Epoch 7/100 - Train Loss: 0.0567, Train Acc: 0.9830, Valid Loss: 0.0973, Valid Acc: 0.9800, Test Loss: 0.0023, Test Acc: 0.9749
Epoch 8/100 - Train Loss: 0.0507, Train Acc: 0.9811, Valid Loss: 0.097

INFO :      Starting Flower simulation, config: num_rounds=19, no round_timeout
INFO:flwr:Starting Flower simulation, config: num_rounds=19, no round_timeout


Epoch 19/100 - Train Loss: 0.0068, Train Acc: 0.9985, Valid Loss: 0.0985, Valid Acc: 0.9850, Test Loss: 0.0021, Test Acc: 0.9814
Early stopping at epoch 19
Early stopping epoch for 8 PCA components: 19
Test Loss: 0.0019
Test Accuracy: 0.9809


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-17 16:09:27,229	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 3932070297.0, 'memory': 7864140596.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3932070297.0, 'memory': 7864140596.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO:fl

(ClientAppActor pid=8456) Expected parameter shapes:
(ClientAppActor pid=8456) conv1.weight: torch.Size([16, 1, 3, 3])
(ClientAppActor pid=8456) conv1.bias: torch.Size([16])
(ClientAppActor pid=8456) conv2.weight: torch.Size([32, 16, 3, 3])
(ClientAppActor pid=8456) conv2.bias: torch.Size([32])
(ClientAppActor pid=8456) fc1.weight: torch.Size([128, 1568])
(ClientAppActor pid=8456) fc1.bias: torch.Size([128])
(ClientAppActor pid=8456) fc2.weight: torch.Size([10, 128])
(ClientAppActor pid=8456) fc2.bias: torch.Size([10])
(ClientAppActor pid=8456) Received parameter shapes:
(ClientAppActor pid=8456) conv1.weight: torch.Size([16, 1, 3, 3])
(ClientAppActor pid=8456) conv1.bias: torch.Size([16])
(ClientAppActor pid=8456) conv2.weight: torch.Size([32, 16, 3, 3])
(ClientAppActor pid=8456) conv2.bias: torch.Size([32])
(ClientAppActor pid=8456) fc1.weight: torch.Size([128, 1568])
(ClientAppActor pid=8456) fc1.bias: torch.Size([128])
(ClientAppActor pid=8456) fc2.weight: torch.Size([10, 128])
(Cl

##data_neglected_percentage = (1 - (n_components / original_dimensions)) * 100


###original_bandwidth_utilization = (original_data_size * num_rounds * num_clients) / total_bandwidth
###reduced_bandwidth_utilization = (reduced_data_size * num_rounds * num_clients) / total_bandwidth
###bandwidth_utilization_reduction = (1 - (reduced_bandwidth_utilization / original_bandwidth_utilization)) * 100

###original_communication_cost = original_data_size * num_rounds * num_clients
###reduced_communication_cost = reduced_data_size * num_rounds * num_clients
###communication_cost_reduction = (1 - (reduced_communication_cost / original_communication_cost)) * 100

###original_latency = original_data_size * transfer_time
###reduced_latency = reduced_data_size * transfer_time
###latency_reduction = (1 - (reduced_latency / original_latency)) * 100